## Gift Contract Testing

In [1]:
from web3.auto import w3
import pandas as pd
from cyberpy import generate_wallet, address_to_address
from cyberpy._message_signer import Message
from cyberpy._wallet import seed_to_privkey
from eth_account.messages import encode_defunct
import json
from subprocess import Popen, PIPE
import os
from time import time
import ipfshttpclient
from tqdm.notebook import tqdm
from base64 import b64encode

client = ipfshttpclient.connect()

NUMBER_OF_PARTICIPANTS = 2
KEY_PHRASE = 'KEY PHRASE'
NICKNAME_LIST = [f'john{round(time())}{number}' for number in range(NUMBER_OF_PARTICIPANTS)]
AVATAR_CID_LIST = [client.add_str(_nickname + 'avatar') for _nickname in NICKNAME_LIST]
CLAIM_AMOUNT_LIST = [1_477_000] * NUMBER_OF_PARTICIPANTS

INITIAL_BALANCE = str(1_000_000_000)
COEF_UP = str(20)
COEF_DOWN = str(5)
TARGET_CLAIM = str(2)

NODE_URL = 'https://rpc.space-pussy-1.cybernode.ai:443'
NETWORK = 'space-pussy-1'

INIT_PASSPORT_CONTRACT = False
PASSPORT_CODE_ID = str(23)
PASSPORT_CONTRACT_ADDRESS = 'bostrom1ttftq90jgl6cms2y0awsatx8rnf99m2u2l6cmdtd427yg6g3myvqgsm4nu'

INIT_GIFT_CONTRACT = False
GIFT_CODE_ID = str(20)
GIFT_CONTRACT_ADDRESS = 'bostrom1rt2acjyhs4jfjdq56pftpu7762hy9gfl63je6fnhwrc5p5y4kmuqxg0262'

ROOT_SOURCE_FILE_NAME = 'root_testing_source_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.json'
ROOT_FILE_NAME = 'root_testing_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses'


def execute_bash(bash_command: str, display_data: bool = False) -> [str, str]:
    _output, _error = Popen(bash_command, shell=True, stdout=PIPE, stderr=PIPE).communicate()
    if _error:
        display_data = True
    if display_data:
        print(bash_command)
        if _output:
            try:
                print(json.dumps(json.loads(_output.decode('utf-8')), indent=4, sort_keys=True))
            except json.JSONDecodeError:
                print(_output)
        if _error:
            print(json.dumps(json.loads(_error.decode('utf-8')), indent=4, sort_keys=True))
    return _output.decode('utf-8'), _error.decode('utf-8')

/Users/sergenedashkovsky/Library/Python/3.8/lib/python/site-packages/ipfshttpclient/client/__init__.py:75: VersionMismatch: Unsupported daemon version '0.10.0' (not in range: 0.5.0 ≤ … < 0.9.0)
  warnings.warn(exceptions.VersionMismatch(version, minimum, maximum))


### Generate addresses and sign messages

In [2]:
claims_list = []
for i in range(NUMBER_OF_PARTICIPANTS):
    claim_item = {'amount': CLAIM_AMOUNT_LIST[i],
                  'nickname': NICKNAME_LIST[i],
                  'avatar': AVATAR_CID_LIST[i]}

    # Generate Ethereum wallet
    ethereum_wallet = w3.eth.account.create(KEY_PHRASE)
    claim_item['ethereum_address'] = ethereum_wallet.address.lower()
    claim_item['ethereum_private_key'] = ethereum_wallet.privateKey.hex()

    # Generate Bostrom and Cosmos wallet
    bostrom_wallet = generate_wallet()
    claim_item['bostrom_address'] = bostrom_wallet['address']
    claim_item['cosmos_address'] = address_to_address(bostrom_wallet['address'], 'cosmos')
    claim_item['cosmos_seed'] = bostrom_wallet['seed']

    # Create message
    # message_json = {
    #     "nickname": claim_item['nickname'],
    #     "gift_claiming_address": claim_item['ethereum_address'],
    #     "gift_amount": claim_item['amount'],
    #     "target_address": claim_item['bostrom_address']
    # }
    # claim_item['message'] = json.dumps(message_json)
    claim_item['message'] = 'QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV'

    # Sign message form Ethereum address
    ethereum_signed_message = \
        w3.eth.account.sign_message(
            signable_message=encode_defunct(text=claim_item['message']),
            private_key=ethereum_wallet.privateKey)
    # print(signed_message.signature)
    claim_item['ethereum_message_signature'] = ethereum_signed_message.signature.hex()

    # Sign message form Cosmos address
    cosmos_msg = Message(privkey=seed_to_privkey(seed=claim_item['cosmos_seed']))
    cosmos_msg.add_message(signing_message=claim_item['message'], signer_prefix='cosmos')
    claim_item['cosmos_message_signed_row'] = cosmos_msg.get_signed_message()
    claim_item['cosmos_message_signature'] = b64encode(json.dumps(claim_item['cosmos_message_signed_row']).replace('\n', '').replace(' ', '').encode('utf-8')).decode("utf-8")

    # Verify message
    assert claim_item['ethereum_address'] == w3.eth.account.recover_message(
        signable_message=encode_defunct(text=claim_item['message']),
        signature=claim_item['ethereum_message_signature']).lower()

    claims_list.append(claim_item)

claims_df = pd.DataFrame(claims_list)
claims_df.to_csv('claims_ethereum_test_data_' + str(NUMBER_OF_PARTICIPANTS) + '_addresses.csv')
claims_df

,amount,nickname,avatar,ethereum_address,ethereum_private_key,bostrom_address,cosmos_address,cosmos_seed,message,ethereum_message_signature,cosmos_message_signed_row,cosmos_message_signature
0,1477000,john16445669770,QmeubEhpA6e8bV8P2Q9fmpcQrkVNcSHYFszcunconYm58v,0x31419fd3943c05836fc708ba6e5b7353accdcc82,0xb5327df33d01f31e0573c5a4484ac6d6313a3e185c40...,bostrom1t6hlkgpkwgx4qx2qjctd6hnzj6emn77zq7ttxs,cosmos1t6hlkgpkwgx4qx2qjctd6hnzj6emn77zrdlcch,layer below poem mom print hidden allow povert...,QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV,0xbaf8c4a43f098c1c2a35ea0513edb361c93ef0113f14...,{'pub_key': 'AiWoSSCPapVl5ob/LHjlQzQkw+kSLTHbc...,eyJwdWJfa2V5IjoiQWlXb1NTQ1BhcFZsNW9iL0xIamxRel...
1,1477000,john16445669771,QmS4pvkZzQcJMDYrMCZKDpBNoWc7HWJnFKHi1dRhmjRrLy,0x42931499d6d14a05fb312808123a564e30a66a0c,0x578a91df8f628383aa92ae7d2a05a0e557c72672e6cf...,bostrom12h0p07w95x5r3xjr03999pvx8535vnemk9d7v3,cosmos12h0p07w95x5r3xjr03999pvx8535vnem4kedjk,lunar calm pond lizard want mobile purpose pat...,QmRX8qYgeZoYM3M5zzQaWEpVFdpin6FvVXvp6RPQK3oufV,0xc37be3a26e4add0a57fde48d3ba130a126e1195350f0...,{'pub_key': 'A1QeouLqZzeXmOicT59AYajB0ynmteSes...,eyJwdWJfa2V5IjoiQTFRZW91THFaemVYbU9pY1Q1OUFZYW...


### Instantiate Passport Contract

In [3]:
if INIT_PASSPORT_CONTRACT:
    init_output, init_error = execute_bash(
        f'''INIT='{{"name":"CPT", "minter":"{os.getenv('WALLET_ADDRESS')}", "owner":"{os.getenv('WALLET_ADDRESS')}", "symbol":"CPT"}}' \
               && cyber tx wasm instantiate {PASSPORT_CODE_ID} "$INIT" --from $WALLET --label "cybergift passport test" -y --gas 3500000 --broadcast-mode block -o json --chain-id={NETWORK} --node={NODE_URL}''')
    if init_error:
        print(init_error)
    init_json = json.loads(init_output)
    passport_contract_address = \
        [event['attributes'][0]['value'] for event in init_json['logs'][0]['events'] if event['type'] == 'instantiate'][0]
else:
    passport_contract_address = PASSPORT_CONTRACT_ADDRESS
print(f'Passport contract address: {passport_contract_address}')

Passport contract address: bostrom1ttftq90jgl6cms2y0awsatx8rnf99m2u2l6cmdtd427yg6g3myvqgsm4nu


### Instantiate Gift Contract

In [4]:
if INIT_GIFT_CONTRACT:
    init_output, init_error = execute_bash(
        f'''INIT='{{"owner":"{os.getenv('WALLET_ADDRESS')}", "passport":"{passport_contract_address}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF_UP}", "target_claim":"{TARGET_CLAIM}"}}' \
               && cyber tx wasm instantiate {GIFT_CODE_ID} "$INIT" --from $WALLET --amount {INITIAL_BALANCE}boot --label "cybergift test" -y --gas 3500000 --broadcast-mode block -o json --chain-id={NETWORK} --node={NODE_URL}''')
    if init_error:
        print(init_error)
    init_json = json.loads(init_output)
    gift_contract_address = \
        [event['attributes'][0]['value'] for event in init_json['logs'][0]['events'] if event['type'] == 'instantiate'][0]
else:
    gift_contract_address = GIFT_CONTRACT_ADDRESS
print(f'Gift contract address: {gift_contract_address}')

Gift contract address: bostrom1rt2acjyhs4jfjdq56pftpu7762hy9gfl63je6fnhwrc5p5y4kmuqxg0262


In [26]:
def get_contract_name(contract_address: str) -> str:
    if contract_address == gift_contract_address:
        return 'Gift Contract'
    elif contract_address == passport_contract_address:
        return 'Passport Contract'
    elif contract_address == os.getenv('WALLET_ADDRESS'):
        return 'Wallet'
    else:
        return contract_address


def parse_contract_execution_json(contract_execution_json: str) -> None:
    for log_item in json.loads(contract_execution_json)['logs']:
        for event_item in log_item['events']:
            print('')
            if event_item['type'] == 'message':
                if len(event_item["attributes"]) == 3:
                    print(f'message from {get_contract_name(event_item["attributes"][-1]["value"])} {event_item["attributes"][1]["value"]} {event_item["attributes"][0]["value"]}')
                else:
                    print(event_item)
            elif event_item['type'] == 'execute':
                if len(event_item["attributes"]) == 1:
                    print(f'execute contract: {get_contract_name(event_item["attributes"][0]["value"])}')
                else:
                    print(event_item)
            elif event_item['type'] == 'cyberlink':
                print('cyberlinks')
                for i in range(len(event_item['attributes'])//2):
                    print(f'\t{event_item["attributes"][i * 2]["value"]} -> {event_item["attributes"][i * 2 + 1]["value"]}')
                print(f'\tneuron: {get_contract_name(event_item["attributes"][-1]["value"])}')
            elif event_item['type'] == 'coin_received':
                print('coin_received')
                for attr_item in event_item["attributes"]:
                    print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'coin_spent':
                print('coin_spent')
                for attr_item in event_item["attributes"]:
                    print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'wasm':
                print('wasm')
                for attr_item in event_item["attributes"]:
                    print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            elif event_item['type'] == 'transfer':
                print('transfer')
                for attr_item in event_item["attributes"]:
                    print(f'\t{attr_item["key"]}: {get_contract_name(attr_item["value"])}')
            else:
                print(event_item)

### Create Merkle Root

In [6]:
root_source_list = [{'address': _item['ethereum_address'],
                     'amount': str(_item['amount'])} for _item in claims_list]+ \
                    [{'address': _item['cosmos_address'],
                      'amount': str(_item['amount'])} for _item in claims_list]

with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
    outfile.write(str(root_source_list).replace("'", '"'))

In [7]:
execute_bash(f'rm {ROOT_FILE_NAME}')
root_output, root_error = execute_bash(
    f'merkle-airdrop-cli generateRoot --file {ROOT_SOURCE_FILE_NAME} >> {ROOT_FILE_NAME} && cat {ROOT_FILE_NAME}')
root = root_output.replace('\n', '')
print(f'Merkle root: {root}')

Merkle root: cec353c5df0b490ab2679ce623deda7205e73280f54a8aaf90e840ba22b1b1c8


In [8]:
root_register_output, root_register_error = \
    execute_bash(
        f'''REGISTER='{{"register_merkle_root":{{"merkle_root":"{root}"}}}}' \
            && CONTRACT="{gift_contract_address}" \
            && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET --broadcast-mode block -y --chain-id={NETWORK} --node={NODE_URL}''')

#### Get Merkle Root form the Gift Contract

In [9]:
print(
    execute_bash(
        f'''QUERY='{{"merkle_root": {{}}}}' \
            && cyber query wasm contract-state smart {gift_contract_address} "$QUERY" --chain-id={NETWORK} --node={NODE_URL}''')[0]
)

data:
  merkle_root: cec353c5df0b490ab2679ce623deda7205e73280f54a8aaf90e840ba22b1b1c8



### Create Passports

In [10]:
def create_passport(claim_row: pd.Series, display_data: bool = False):
    return execute_bash(
                f'''REGISTER='{{"create_passport":{{"avatar":"{claim_row["avatar"]}", "nickname":"{claim_row["nickname"]}"}}}}' \
                    && CONTRACT="{passport_contract_address}" \
                    && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET -o json --broadcast-mode block -y --gas=400000 --chain-id={NETWORK} --node={NODE_URL}''',
                display_data=display_data)

for index, row in tqdm(claims_df.iterrows()):
    if index == 0:
        create_passport_json, _ = create_passport(row, display_data=True)
    else:
        create_passport(row)

0it [00:00, ?it/s]

REGISTER='{"create_passport":{"avatar":"QmeubEhpA6e8bV8P2Q9fmpcQrkVNcSHYFszcunconYm58v", "nickname":"john16445669770"}}'                     && CONTRACT="bostrom1ttftq90jgl6cms2y0awsatx8rnf99m2u2l6cmdtd427yg6g3myvqgsm4nu"                     && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET -o json --broadcast-mode block -y --gas=400000 --chain-id=space-pussy-1 --node=https://rpc.space-pussy-1.cybernode.ai:443
{
    "code": 0,
    "codespace": "",
    "data": "0A260A242F636F736D7761736D2E7761736D2E76312E4D736745786563757465436F6E7472616374",
    "gas_used": "237661",
    "gas_wanted": "400000",
    "height": "1224329",
    "info": "",
    "logs": [
        {
            "events": [
                {
                    "attributes": [
                        {
                            "key": "particleFrom",
                            "value": "QmNbQbcUooRXS5gepU1M78qGUKE25x7aJfo6GKTEWn9amf"
                        },
                        {
                           

In [27]:
parse_contract_execution_json(create_passport_json)


cyberlinks
	QmNbQbcUooRXS5gepU1M78qGUKE25x7aJfo6GKTEWn9amf -> QmeT4VmXf4nP2i5UJ9K54HbGr5ZTH3rZnW7bwtmuNrsqkX
	QmNbQbcUooRXS5gepU1M78qGUKE25x7aJfo6GKTEWn9amf -> QmeubEhpA6e8bV8P2Q9fmpcQrkVNcSHYFszcunconYm58v
	QmeT4VmXf4nP2i5UJ9K54HbGr5ZTH3rZnW7bwtmuNrsqkX -> QmNbQbcUooRXS5gepU1M78qGUKE25x7aJfo6GKTEWn9amf
	QmeubEhpA6e8bV8P2Q9fmpcQrkVNcSHYFszcunconYm58v -> QmNbQbcUooRXS5gepU1M78qGUKE25x7aJfo6GKTEWn9amf
	neuron: Passport Contract

execute contract: Passport Contract

message from Wallet wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: mint
	minter: Passport Contract
	token_id: 11


### Proof (Add) Address to Passports

In [12]:
def proof_address(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    return execute_bash(
            f'''REGISTER='{{"proof_address":{{"nickname":"{claim_row["nickname"]}", "address":"{claim_row[network + "_address"]}", "signature":"{claim_row[network + "_message_signature"]}"}}}}' \
                && CONTRACT="{passport_contract_address}" \
                && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET -o json --broadcast-mode block -y --gas=400000 --chain-id={NETWORK} --node={NODE_URL}''',
            display_data=display_data)

for index, row in tqdm(claims_df.iterrows()):
    if index == 1:
        proof_ethereum_address_json, _ = proof_address(row, display_data=True)
        proof_cosmos_address_json, _ = proof_address(row, network='cosmos', display_data=True)
    else:
        proof_address(row)
        proof_address(row, network='cosmos', display_data=True)

0it [00:00, ?it/s]

REGISTER='{"proof_address":{"nickname":"john16445669770", "address":"cosmos1t6hlkgpkwgx4qx2qjctd6hnzj6emn77zrdlcch", "signature":"eyJwdWJfa2V5IjoiQWlXb1NTQ1BhcFZsNW9iL0xIamxRelFrdytrU0xUSGJjdVR2emp5TStTVUsiLCJzaWduYXR1cmUiOiJJZUt0WmRLSE1uKy9uV0V5YkVLWjBTbS8yL0x0YVIzNGFaQzY2emZ1Q1hBTW5mN1pYV2hXUlgzYWNEbkNFVk1jVEZYVUQ2Z2NqdnNZdndCeG95NFN5QT09In0="}}'                 && CONTRACT="bostrom1ttftq90jgl6cms2y0awsatx8rnf99m2u2l6cmdtd427yg6g3myvqgsm4nu"                 && cyber tx wasm execute $CONTRACT "$REGISTER" --from $WALLET -o json --broadcast-mode block -y --gas=400000 --chain-id=space-pussy-1 --node=https://rpc.space-pussy-1.cybernode.ai:443
{
    "code": 0,
    "codespace": "",
    "data": "0A260A242F636F736D7761736D2E7761736D2E76312E4D736745786563757465436F6E7472616374",
    "gas_used": "206533",
    "gas_wanted": "400000",
    "height": "1224332",
    "info": "",
    "logs": [
        {
            "events": [
                {
                    "attributes": [
                     

In [28]:
parse_contract_execution_json(proof_ethereum_address_json)


cyberlinks
	QmNbQbcUooRXS5gepU1M78qGUKE25x7aJfo6GKTEWn9amf -> QmPbwkom19D6txb69sfz2x4UznTYnJ8jKxqFhwUPnXeMri
	QmPbwkom19D6txb69sfz2x4UznTYnJ8jKxqFhwUPnXeMri -> QmNbQbcUooRXS5gepU1M78qGUKE25x7aJfo6GKTEWn9amf
	neuron: Passport Contract

execute contract: Passport Contract

message from Wallet wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16445669771
	address: 0x42931499d6d14a05fb312808123a564e30a66a0c


In [29]:
parse_contract_execution_json(proof_cosmos_address_json)


cyberlinks
	QmNbQbcUooRXS5gepU1M78qGUKE25x7aJfo6GKTEWn9amf -> QmWJy7gCB6qp1YcsJLaY8PbodwnQvVXjGmSAedNJdEB6uR
	QmWJy7gCB6qp1YcsJLaY8PbodwnQvVXjGmSAedNJdEB6uR -> QmNbQbcUooRXS5gepU1M78qGUKE25x7aJfo6GKTEWn9amf
	neuron: Passport Contract

execute contract: Passport Contract

message from Wallet wasm /cosmwasm.wasm.v1.MsgExecuteContract

wasm
	_contract_address: Passport Contract
	action: proof_address
	nickname: john16445669771
	address: cosmos12h0p07w95x5r3xjr03999pvx8535vnem4kedjk


### Generate Proofs and Claim

In [15]:
def generate_proof_and_claim(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    proof_output, _ = execute_bash(f'merkle-airdrop-cli generateProofs --file {ROOT_SOURCE_FILE_NAME} --address {claim_row[network + "_address"]} --amount {claim_row.amount}',
                                   display_data=display_data)
    proof = proof_output.replace('\n','').replace(" ",'').replace("'", '"')
    return execute_bash(
                f'''CLAIM='{{"claim":{{"nickname":"{claim_row['nickname']}", "gift_claiming_address":"{claim_row[network + "_address"]}", "gift_amount":"{claim_row.amount}", "proof":{proof}}}}}' \
                    && cyber tx wasm execute {gift_contract_address} "$CLAIM" --from $WALLET -o json --broadcast-mode block -y --gas 400000 --chain-id={NETWORK} --node={NODE_URL}''',
                display_data=display_data)

for index, row in tqdm(claims_df.iterrows()):
    if index == 1:
        claim_ethereum_json, _ = generate_proof_and_claim(row, display_data=True)
        claim_cosmos_json, _ = generate_proof_and_claim(row, network='cosmos', display_data=True)
    else:
        generate_proof_and_claim(row)
        generate_proof_and_claim(row, network='cosmos')

0it [00:00, ?it/s]

merkle-airdrop-cli generateProofs --file root_testing_source_2_addresses.json --address 0x42931499d6d14a05fb312808123a564e30a66a0c --amount 1477000
b"[\n  'b9bf7453f7b3442c281fd3c3ffa2aec387395682c0d16cb6800f8ca06c407e44',\n  '8b39367c8d583dc84ff9adf8dc57424b28520bb7dfd3e2b001370c22b5d12626'\n]\n"
CLAIM='{"claim":{"nickname":"john16445669771", "gift_claiming_address":"0x42931499d6d14a05fb312808123a564e30a66a0c", "gift_amount":"1477000", "proof":["b9bf7453f7b3442c281fd3c3ffa2aec387395682c0d16cb6800f8ca06c407e44","8b39367c8d583dc84ff9adf8dc57424b28520bb7dfd3e2b001370c22b5d12626"]}}'                     && cyber tx wasm execute bostrom1rt2acjyhs4jfjdq56pftpu7762hy9gfl63je6fnhwrc5p5y4kmuqxg0262 "$CLAIM" --from $WALLET -o json --broadcast-mode block -y --gas 400000 --chain-id=space-pussy-1 --node=https://rpc.space-pussy-1.cybernode.ai:443
{
    "code": 0,
    "codespace": "",
    "data": "0A260A242F636F736D7761736D2E7761736D2E76312E4D736745786563757465436F6E7472616374",
    "gas_used": "308

In [30]:
parse_contract_execution_json(claim_ethereum_json)


coin_received
	receiver: Wallet
	amount: 100000boot

coin_spent
	spender: Gift Contract
	amount: 100000boot

execute contract: Gift Contract

message from Wallet wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: Wallet
	sender: Gift Contract
	amount: 100000boot

wasm
	_contract_address: Gift Contract
	action: claim
	original: 0x42931499d6d14a05fb312808123a564e30a66a0c
	target: Wallet
	amount: 28885541


In [31]:
parse_contract_execution_json(claim_cosmos_json)


coin_received
	receiver: Wallet
	amount: 100000boot

coin_spent
	spender: Gift Contract
	amount: 100000boot

execute contract: Gift Contract

message from Wallet wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: Wallet
	sender: Gift Contract
	amount: 100000boot

wasm
	_contract_address: Gift Contract
	action: claim
	original: cosmos12h0p07w95x5r3xjr03999pvx8535vnem4kedjk
	target: Wallet
	amount: 28231082


### Release Gift

In [18]:
def release(claim_row: pd.Series, network: str = 'ethereum', display_data: bool = False):
    return execute_bash(
                f'''CLAIM='{{"release":{{"gift_address":"{claim_row[network + "_address"]}"}}}}' \
                    && cyber tx wasm execute {gift_contract_address} "$CLAIM" --from $WALLET -o json --broadcast-mode block -y --gas 400000 --chain-id={NETWORK} --node={NODE_URL}''',
                display_data=display_data)

for index, row in tqdm(claims_df.iterrows()):
    if index == 1:
        release_ethereum_json, _ = release(row, display_data=True)
        release_cosmos_json, _ = release(row, network='cosmos', display_data=True)
    else:
        release(row)
        release(row, network='cosmos')


0it [00:00, ?it/s]

CLAIM='{"release":{"gift_address":"0x42931499d6d14a05fb312808123a564e30a66a0c"}}'                     && cyber tx wasm execute bostrom1rt2acjyhs4jfjdq56pftpu7762hy9gfl63je6fnhwrc5p5y4kmuqxg0262 "$CLAIM" --from $WALLET -o json --broadcast-mode block -y --gas 400000 --chain-id=space-pussy-1 --node=https://rpc.space-pussy-1.cybernode.ai:443
{
    "code": 0,
    "codespace": "",
    "data": "0A260A242F636F736D7761736D2E7761736D2E76312E4D736745786563757465436F6E7472616374",
    "gas_used": "159137",
    "gas_wanted": "400000",
    "height": "1224341",
    "info": "",
    "logs": [
        {
            "events": [
                {
                    "attributes": [
                        {
                            "key": "receiver",
                            "value": "bostrom135ca8hdpy9sk0ntwqzpzsvatyl48ptx5j359lz"
                        },
                        {
                            "key": "amount",
                            "value": "2878554boot"
                     

In [32]:
parse_contract_execution_json(release_ethereum_json)


coin_received
	receiver: Wallet
	amount: 2878554boot

coin_spent
	spender: Gift Contract
	amount: 2878554boot

execute contract: Gift Contract

message from Wallet wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: Wallet
	sender: Gift Contract
	amount: 2878554boot

wasm
	_contract_address: Gift Contract
	action: release
	address: Wallet
	gift_address: 0x42931499d6d14a05fb312808123a564e30a66a0c
	stage: 9
	amount: 2878554


In [33]:
parse_contract_execution_json(release_cosmos_json)


coin_received
	receiver: Wallet
	amount: 2813108boot

coin_spent
	spender: Gift Contract
	amount: 2813108boot

execute contract: Gift Contract

message from Wallet wasm /cosmwasm.wasm.v1.MsgExecuteContract

transfer
	recipient: Wallet
	sender: Gift Contract
	amount: 2813108boot

wasm
	_contract_address: Gift Contract
	action: release
	address: Wallet
	gift_address: cosmos12h0p07w95x5r3xjr03999pvx8535vnem4kedjk
	stage: 9
	amount: 2813108


In [34]:
# gift_address  0x30f99350a8b2e4acf87cdb197e347fb434fe0856
# gift amount   1_477_000
# first claim  28_885_541
# claim transfer  100_000
# first release 2_878_554

print(f'first release: {int((28_885_541 - 100_000) * 0.1):>_}')
print(f'first coef: {28_885_541 / 1_477_000:>,.2f}')


first release: 2_878_554
coef: 19.56
